In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.colors as colors

def grabFileSeries(scratchdirectory, fn, f0=0, step=1, width=5, 
                   basename="MPsimple", scratchPath="/mnt/gs21/scratch/freem386/"):
    """Returns a list of files with the form 
    <scratchPath>/<scratchdirectory>/<basename>.out2/#####.athdf,
    starting at f0, ending at fn with stepsize step.

    Args:
        scratchdirectory (str): Directory within scratchPath of desired files
        fn (int): final (stopping point) output number
        f0 (int, optional): initial output number. Defaults to 0.
        step (int, optional): output number step. Defaults to 1.
        width (int, optional): width of 0 in filenames. Defaults to 5.
        basename (str, optional): <basename>.out2.#####.athdf . Defaults to "MPsimple".
        scratchPath (str, optional): value stored in $SCRATCH. Defaults to "/mnt/gs21/scratch/freem386/".

    Returns:
        array[str]: List of filenames including paths
    """
    files = []
    for f in np.arange(f0, fn+1, step, dtype=int):
            files.append(scratchPath + scratchdirectory + basename + ".out2." + str(f).zfill(width) + ".athdf")
    return files


In [2]:
filehandler = open("lhlld2_stability.pkl", "rb")
pspace = pickle.load(filehandler)
filehandler.close()

In [4]:
ind = {"ialpha": 0, "ibeta": 1, "ip": 2, "id": 3, "im": 4}  # Dictionary for referencing given values within the pickled data
ti = 0
pi, di, mi = 0, 0, 0    # For easy changing

# Define the parameters you want to vary
alpha_values = np.array([0, 1, 3, 5, 7])
beta_values = np.array([0, 1, 2, 3, 5])
pcoeff_values = np.array([1e-8, 1e-7, 1e-6])
d_values = np.array([1.218085e-5, 1.218085e-4, 1.218085e-3])
b_values = np.array([5e4, 5e5, 5e6])

for ti in np.arange(start=11, step=1, stop=12):
    for pi in range(3):
        for di in range(3):
            for mi in range(3):
                print(pi, di, mi, ti)
                fig, ax = plt.subplots(2, 2)

                X, Y = np.meshgrid(alpha_values, beta_values)
                Z = pspace[:, :, pi, di, mi, ti]
                pcm = ax[0,0].pcolormesh(X, Y, Z, norm=colors.LogNorm(vmin=np.nanmin(Z), vmax=np.nanmax(Z)))

                ax[0,0].set_xlabel(r"$\alpha$")
                ax[0,0].set_ylabel(r"$\beta$")
                fig.colorbar(pcm, ax=ax[0,0], extend="max")

                r = np.linspace(0, 2, 100)
                rho = d_values[di]*r**alpha_values[:, np.newaxis]
                p = 4*np.pi*pcoeff_values[pi] * r**(2*beta_values[:, np.newaxis]) * 2*b_values[mi]**2
                bp1 = beta_values + np.ones_like(beta_values)
                bm1 = beta_values - np.ones_like(beta_values)
                b_numerator = b_values[mi]*r**(bp1[:,np.newaxis])
                with np.errstate(divide='ignore', invalid='ignore'):
                    b = b_numerator/bm1[:,np.newaxis]

                for i in range(b.shape[0]):
                    ax[1, 0].loglog(r, b[i, :], '--', label=f'B-be={beta_values[i]}')
                ax[1, 0].legend()

                for i in range(rho.shape[0]):
                    ax[1, 1].loglog(r, rho[i,:], '--', label=f'rho-al={alpha_values[i]}')
                ax[1, 1].legend()

                for i in range(p.shape[0]):
                    ax[0, 1].loglog(r, p[i, :], '--', label=f'p-be={beta_values[i]}')
                ax[0, 1].legend()

                fig.set_label(f"t{ti}_p{pi}_d{di}_b{mi}")
                fig.tight_layout()
                fig.suptitle(f"t{ti}_p{pi}_d{di}_b{mi}")

                plt.savefig(f"plot3dmin/t{ti:03}_p{pi}_d{di}_b{mi}.png")

                plt.close()

0 0 0 11
0 0 1 11
0 0 2 11
0 1 0 11
0 1 1 11
0 1 2 11
0 2 0 11
0 2 1 11
0 2 2 11
1 0 0 11
1 0 1 11
1 0 2 11
1 1 0 11
1 1 1 11
1 1 2 11
1 2 0 11
1 2 1 11
1 2 2 11
2 0 0 11
2 0 1 11
2 0 2 11
2 1 0 11
2 1 1 11
2 1 2 11
2 2 0 11
2 2 1 11
2 2 2 11


In [ ]:
ffmpeg -framerate 2 -i t%03d_p0_d0_b0.png p0_d0_b0.mp4